# EDA та фізика сигналу для детекції GNSS-атак

Цей ноутбук містить розвідувальний аналіз та фізичні перевірки для задачі детекції атак на GNSS. Дані: `data/processed/all_data_compressed.parquet`.

## 1. Setup & Loading
Завантаження даних, оптимізація пам'яті та конвертація часу.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook')

# Створюємо папку для графіків
os.makedirs('figures/eda', exist_ok=True)

# Завантаження даних з оптимізацією пам'яті
try:
    df = pd.read_parquet('data/processed/all_data_compressed.parquet')
    for col in ['cnoMean', 'numSV', 'numSatsTracked', 'hAcc', 'hDOP', 'vDOP']:
        if col in df.columns:
            df[col] = df[col].astype('float32')
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    print('Дані успішно завантажено!')
except Exception as e:
    print(f'Помилка при завантаженні: {e}')

Помилка при завантаженні: [Errno 2] No such file or directory: 'data/processed/all_data_compressed.parquet'


## 2. Basic Signal Physics (Базова фізика)
### Розподіл CNO (Carrier-to-Noise Ratio)
Цей графік дозволяє побачити бімодальність: одна мода — нормальний сигнал, інша — атака або деградація.

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df['cnoMean'], bins=60, kde=True, color='royalblue')
plt.title('Розподіл CNO (Carrier-to-Noise Ratio)')
plt.xlabel('cnoMean (dBHz)')
plt.ylabel('Кількість')
plt.tight_layout()
plt.savefig('figures/eda/cnoMean_hist.png', dpi=300)
plt.show()

### Залежність точності від сили сигналу (Scatter Plot)
Цей графік доводить, що при CNO < 25 точність падає експоненційно (ефект 'клюшки').

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df['cnoMean'], df['hAcc'], alpha=0.2, s=2, color='grey')
plt.yscale('log')
plt.xlabel('cnoMean (dBHz)')
plt.ylabel('hAcc (мм, log scale)')
plt.title('Залежність точності від сили сигналу (CNO vs hAcc)')
plt.tight_layout()
plt.savefig('figures/eda/cnoMean_vs_hAcc_scatter.png', dpi=300)
plt.show()

### Ефективність використання супутників (Hexbin Plot)
Hexbin-графік показує, як часто кількість видимих супутників збігається з кількістю використаних. В ідеалі — діагональ.

In [ ]:
plt.figure(figsize=(7,6))
plt.hexbin(df['numSV'], df['numSatsTracked'], gridsize=40, cmap='Blues', mincnt=1)
plt.plot([df['numSV'].min(), df['numSV'].max()], [df['numSV'].min(), df['numSV'].max()], 'r--', lw=2, label='Ідеал')
plt.xlabel('numSV (Видимі супутники)')
plt.ylabel('numSatsTracked (Використані супутники)')
plt.title('Hexbin: numSV vs numSatsTracked')
plt.colorbar(label='Кількість')
plt.legend()
plt.tight_layout()
plt.savefig('figures/eda/hexbin_numSV_numSatsTracked.png', dpi=300)
plt.show()

## 3. Advanced Dynamics (Динаміка переходів)
### Аналіз переходів 'Норма' → 'Атака'
Визначаємо моменти переходу на основі порогу hAcc > 50000 мм. Досліджуємо, як змінюється ефективність супутників при вході та виході з атаки.

In [ ]:
# Додаємо sat_efficiency
df['sat_efficiency'] = (df['numSV'] / df['numSatsTracked'].replace(0, 1)).clip(0, 5)

# Визначаємо атаку
attack = (df['hAcc'] > 50000).astype(int)
transitions = attack.diff().fillna(0)
attack_start_idx = transitions[transitions == 1].index
attack_end_idx = transitions[transitions == -1].index

# Гістерезис: як поводиться sat_efficiency при вході/виході з атаки
window = 100 # точок до/після переходу
loops = []
for idx in attack_start_idx:
    if idx > window and idx+window < len(df):
        loops.append(df['sat_efficiency'].iloc[idx-window:idx+window].values)

plt.figure(figsize=(10,6))
for loop in loops[:30]:
    plt.plot(np.arange(-window, window), loop, alpha=0.2, color='blue')
plt.title('Гістерезис: sat_efficiency навколо переходу в атаку')
plt.xlabel('Індекс відносно переходу')
plt.ylabel('sat_efficiency')
plt.tight_layout()
plt.savefig('figures/eda/hysteresis_sat_efficiency.png', dpi=300)
plt.show()

# Slope Analysis: швидкість деградації
slopes = []
for idx in attack_start_idx:
    if idx > 10:
        y = df['hAcc'].iloc[idx-10:idx].values

## 4. Reliability Analysis (Аналіз надійності)
### TTFF (Time to First Failure)
Розбиваємо дані на сесії польотів, рахуємо час до першого збою (hAcc > 50000).

In [ ]:
# Сесії: якщо різниця між timestamp > 60 c, це новий політ
df = df.sort_values('timestamp').reset_index(drop=True)
dt = df['timestamp'].diff().dt.total_seconds().fillna(0)
session_id = (dt > 60).cumsum()
df['session_id'] = session_id

ttff_list = []
for sid, group in df.groupby('session_id'):

## 5. Anomaly Detection (Пошук аномалій)
### DOP-матриця кореляції та аналіз 'мертвих нулів'
Перевіряємо кореляцію між HDOP, VDOP, PDOP та аналізуємо частоту появи CNO = 0.

In [ ]:
# DOP-матриця
dop_cols = [col for col in ['hDOP', 'vDOP', 'pDOP'] if col in df.columns]
if len(dop_cols) > 1:
    plt.figure(figsize=(5,4))
    corr = df[dop_cols].corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('DOP-матриця кореляції')
    plt.tight_layout()
    plt.savefig('figures/eda/dop_corr_matrix.png', dpi=300)
    plt.show()

# Аналіз мертвих нулів
plt.figure(figsize=(7,4))
zero_count = (df['cnoMean'] == 0).sum()
sns.histplot(df['cnoMean'][df['cnoMean'] <= 10], bins=20, color='red')
plt.title(f'Аналіз мертвих нулів (CNO=0): {zero_count} точок')
plt.xlabel('cnoMean (dBHz)')
plt.tight_layout()
plt.savefig('figures/eda/dead_zeros_cnoMean.png', dpi=300)
plt.show()